# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a hospital. Specifically, this report will be targeted to stakeholders interested in opening a **small hospital** in **Merida, Yucatan**, Mexico. Since this small hospital will be competing with other hospitals and doctor's offices, we will refer as "hospital" to the sum of hospitals plus doctor's offices in a given area. 

Since there are many medical centers in Merida we will try to detect **locations that are not already crowded with medical centers**. We are also particularly interested in **areas with no hospitals in vicinity**. We would also prefer locations **north to city center**, assuming that first two conditions are met.

We will generate a few most promising neighborhoods based on these criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* Number of existing medical centers in the neighborhood (any type of medical center)
* Number of and distance to hospitals in the neighborhood, if any
* Distance of neighborhood from city center in the north direction (just as a reference)

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* Centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Geocoder library reverse geocoding**
* Number of medical centers and their type and location in every neighborhood will be obtained using **Foursquare API**
* Coordinates of Merida center will be obtained using **Geocoder library** of well known Merida location (Merida Centro)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest, which is approx. 24x24 kilometers centered around Merida city center.

Let's first find the latitude & longitude of Merida city center, using specific, well-known address and Geocoder library.

In [1]:
import geocoder
g = geocoder.arcgis('Merida, YUC')
latitude = g.latlng[0]
longitude = g.latlng[1]
merida_center = latitude, longitude
print('Merida, Yucatan coordinates are: {}, {}.'.format(g.latlng[0], g.latlng[1]))
merida_center

Merida, Yucatan coordinates are: 20.96670000000006, -89.62316999999996.


(20.96670000000006, -89.62316999999996)

Now let's create a grid of area candidates, equally spaced, centered around city center and within **~12km** from Merida Centro, which covers the entire city and some adjacent inhabited areas. Our neighborhoods will be defined as circular areas with a radius of 600 meters, so our neighborhood centers will be 1200 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [2]:
#!pip install pyproj
import pyproj
from pyproj import Transformer

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.crs.CRS(proj='latlong',datum='WGS84')
    proj_xy = pyproj.crs.CRS(proj="utm", zone=16, datum='WGS84')
    #xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    #return xy[0], xy[1]
    t = Transformer.from_crs(proj_latlon, proj_xy, always_xy=True)
    x, y = t.transform(lon, lat)
    return x, y

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.crs.CRS(proj='latlong',datum='WGS84')
    proj_xy = pyproj.crs.CRS(proj="utm", zone=16, datum='WGS84')
    #lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    #return lonlat[0], lonlat[1]
    t1 = Transformer.from_crs(proj_xy, proj_latlon, always_xy=True)
    lonlat = t1.transform(x, y)
    return lonlat[0], lonlat[1]
        
def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Merida center longitude={}, latitude={}'.format(merida_center[1], merida_center[0]))
x, y = lonlat_to_xy(merida_center[1], merida_center[0])
print('Merida center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Merida center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Merida center longitude=-89.62316999999996, latitude=20.96670000000006
Merida center UTM X=227245.26125269826, Y=2320697.5036618053
Merida center longitude=-89.62316999999996, latitude=20.96670000000006


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [3]:
merida_center_x, merida_center_y = lonlat_to_xy(merida_center[1], merida_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = merida_center_x - 12000
x_step = 1200
y_min = merida_center_y - 12000 - (int(42/k)*k*1200 - 12000)/2
y_step = 1200 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(63/k)):
    y = y_min + i * y_step
    x_offset = 600 if i%2==0 else 0
    for j in range(0, 63):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(merida_center_x, merida_center_y, x, y)
        if (distance_from_center <= 12001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

360 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [4]:
#!pip install folium
import folium

map_merida = folium.Map(location=merida_center, zoom_start=12)
folium.Marker(merida_center, popup='Merida Centro').add_to(map_merida)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=600, color='blue', fill=False).add_to(map_merida)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_merida

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within **~12km from Merida Centro**. 

Let's now use Geocoder library to get approximate addresses of those locations.

In [5]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    geo_address = geocoder.arcgis([lat, lon], method = 'reverse')
    address = geo_address.address
    if address is None:
        address = 'NO ADDRESS'
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [6]:
#take a look at adresses format
addresses[150:170]

['Calle 6, Poligono San Pedro, Kanasín, Yucatán, 97370',
 'Calle 23G 406-412, Los Encinos, Kanasín, Yucatán, 97370',
 'Kanasín, Yucatán',
 'Kanasín, Yucatán',
 'Kanasín, Yucatán',
 'Mérida, Yucatán',
 'Mérida, Yucatán',
 'Mérida, Yucatán',
 '97249',
 'Calle 65A, Fraccionamiento Juan Pablo II, Mérida, Yucatán, 97246',
 'Adventista del 7mo Día',
 'Calle 71A 885, Fracc Jardines de Nueva Mulsay, Mérida, Yucatán, 97246',
 'Calle 75 614A, Sambula, Mérida, Yucatán, 97250',
 'Calle 74 561B, Fracc Jardines de San Sebastian, Mérida, Yucatán, 97000',
 'Calle 58 606, Mérida Centro, Mérida, Yucatán, 97000',
 'Calle 69 379A, Azcorra, Mérida, Yucatán, 97000',
 'Calle 27 133J, Jardines de Miraflores, Mérida, Yucatán, 97168',
 'Calle 65 703, Fraccionamiento Emilio Portes Gil, Mérida, Yucatán, 97167',
 'Calle 27 550-550, Nueva Chichen-Itza, Mérida, Yucatán, 97170',
 'Calle 53 369, Fraccionamiento Bosques de Oriente, Mérida, Yucatán, 97178']

Let's now place all this into a Pandas dataframe.

In [7]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Diagonal 24, Dzununcán, Mérida, Yucatán, 97315",20.865108,-89.655983,223645.261253,2.309501e+06,11760.689991
1,"Mérida, Yucatán",20.865286,-89.644459,224845.261253,2.309501e+06,11450.494709
2,"Privada 56, Xmatkuil, Mérida, Yucatán, 97315",20.865464,-89.632935,226045.261253,2.309501e+06,11260.276598
3,"Mérida, Yucatán",20.865641,-89.621411,227245.261253,2.309501e+06,11196.152423
4,"Mérida, Yucatán",20.865817,-89.609887,228445.261253,2.309501e+06,11260.276598
5,"Mérida, Yucatán",20.865993,-89.598363,229645.261253,2.309501e+06,11450.494709
6,"Mérida, Yucatán",20.866167,-89.586839,230845.261253,2.309501e+06,11760.689991
7,"Mérida, Yucatán",20.874218,-89.673434,221845.261253,2.310541e+06,11503.176225
8,"Mérida, Yucatán",20.874398,-89.661910,223045.261253,2.310541e+06,10991.044685
9,"Mérida, Yucatán",20.874577,-89.650385,224245.261253,2.310541e+06,10590.706457


In [8]:
# check for duplicates and null values
print(df_locations[df_locations.duplicated(['Address'], keep = 'first')])
print(df_locations['Address'].isnull().sum())

                                               Address   Latitude  Longitude  \
3                                      Mérida, Yucatán  20.865641 -89.621411   
4                                      Mérida, Yucatán  20.865817 -89.609887   
5                                      Mérida, Yucatán  20.865993 -89.598363   
6                                      Mérida, Yucatán  20.866167 -89.586839   
7                                      Mérida, Yucatán  20.874218 -89.673434   
..                                                 ...        ...        ...   
351  Carretera Cheumán-Cholul, Temozón Norte, Mérid...  21.062893 -89.607540   
353                                    Mérida, Yucatán  21.063246 -89.584462   
354                                    Mérida, Yucatán  21.063422 -89.572922   
358                     Temozón Norte, Mérida, Yucatán  21.072185 -89.613474   
359                     Temozón Norte, Mérida, Yucatán  21.072362 -89.601934   

                 X             Y  Dista

...and let's now save/persist this data into local file.

In [9]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on medical centers in each neighborhood.

We're interested in venues in 'medical center' category, but only those that are proper medical centers - veterinaries, quiropractors, medical labs, etc. are not direct competitors so we don't care about those. So we will include in our list only venues that have some key words in category name, and we'll make sure to detect and include all the subcategories of specific 'hospital' category (hospital and doctor's office), as we need info on hospitals in the neighborhood.

Foursquare credentials are defined in hidden config file.

In [10]:
import config
CLIENT_ID = config.CLIENT_ID # Foursquare ID
CLIENT_SECRET = config.CLIENT_SECRET # Foursquare Secret

In [11]:
# Category IDs corresponding to Medical Centers were taken from Foursquare web site (https://developer.foursquare.com/docs/build-with-foursquare/categories):

import requests

medical_category = '4bf58dd8d48988d104941735' # 'Root' category for all medical center venues

hospital_categories = ['4bf58dd8d48988d177941735','4bf58dd8d48988d196941735'] #Doctor's Office, Hospital

#In "hospital" we include doctor's offices 

def is_hospital(categories, specific_filter=None):
    medical_words = ['office', 'doctor', 'hospital', 'ward', 'clinic', 'therapist', 'center']
    medical = False
    hospital = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for h in medical_words:
            if h in category_name:
                medical = True
        if 'lab' in category_name:
            medical = False
        if not(specific_filter is None) and (category_id in specific_filter):
            hospital = True
            medical = True
        if "Dentist's" in category_name:
            hospital = False
    return medical, hospital

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=700, limit=100):
    version = '20210515'
    #query='hospital'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&categoryId={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        category,
        #query,
        lat, 
        lon,
        version,
        radius, 
        limit)
    
    results = requests.get(url).json()['response']['groups'][0]['items']

    venues = [(item['venue']['id'],
               item['venue']['name'],
               get_categories(item['venue']['categories']),
               (item['venue']['location']['lat'], item['venue']['location']['lng']),
               format_address(item['venue']['location']),
               item['venue']['location']['distance']) for item in results]        

    return venues

In [12]:
# Let's now go over our neighborhood locations and get nearby HOSPITALS; we'll also maintain a dictionary of all found medical centers and all found hospitals

import pickle

def get_medical(lats, lons):
    medical_centers = {}
    hospitals = {}
    location_medicals = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=700 to make sure we have overlaps/full coverage so we don't miss any medical center (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, medical_category, CLIENT_ID, CLIENT_SECRET, radius=700, limit=100)
        area_medicals = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_med, is_hosp = is_hospital(venue_categories, specific_filter=hospital_categories)
            if is_med:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                medical = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_hosp, x, y)
                if venue_distance<=300:
                    area_medicals.append(medical)
                medical_centers[venue_id] = medical
                if is_hosp:
                    hospitals[venue_id] = medical
        location_medicals.append(area_medicals)
        print(' .', end='')
    print(' done.')
    return medical_centers, hospitals, location_medicals

# Try to load from local file system in case we did this before
medical_centers = {}
hospitals = {}
location_medicals = []
loaded = False
try:
    with open('medical_centers_700.pkl', 'rb') as f:
        medical_centers = pickle.load(f)
    with open('hospitals_700.pkl', 'rb') as f:
        hospitals = pickle.load(f)
    with open('location_medicals_700.pkl', 'rb') as f:
        location_medicals = pickle.load(f)
    print('Hospital data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    medical_centers, hospitals, location_medicals = get_medical(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('medical_centers_700.pkl', 'wb') as f:
        pickle.dump(medical_centers, f)
    with open('hospitals_700.pkl', 'wb') as f:
        pickle.dump(hospitals, f)
    with open('location_medicals_700.pkl', 'wb') as f:
        pickle.dump(location_medicals, f)
        

Hospital data loaded.


In [13]:
import numpy as np

print('Total number of medical centers:', len(medical_centers))
print('Total number of hospitals:', len(hospitals))
print('Percentage of hospitals: {:.2f}%'.format(len(hospitals) / len(medical_centers) * 100))
print('Average number of medical centers in neighborhood:', np.array([len(h) for h in location_medicals]).mean())

Total number of medical centers: 547
Total number of hospitals: 258
Percentage of hospitals: 47.17%
Average number of medical centers in neighborhood: 0.2777777777777778


In [14]:
print('List of all medical centers')
print('-----------------------')
for r in list(medical_centers.values())[:10]:
    print(r)
print('...')
print('Total:', len(medical_centers))

List of all medical centers
-----------------------
('5aa96c8ad7627e7413362a29', 'Villas De Transición Hospitalaria', 20.903318, -89.64109, 'México', 422, True, 225265.2624593384, 2313707.9749038783)
('4dc085c06e8124d25dcf5661', 'Hospital de la Amistad Corea-México', 20.902425049940813, -89.63986104659945, 'Calle 60 (San José Tecoh II), Mérida, Yucatán, México', 655, True, 225391.5431065217, 2313606.967368572)
('52ea95d4498e0eeef0685f29', 'Unidad de Hospitalizacion CIJ', 20.902143, -89.635632, 'México', 232, True, 225831.1848048609, 2313568.4929117775)
('5a136f23a22db7017a9ccf25', 'Consultorio Medico Best', 20.908412, -89.64842, '97297 Mérida, Yucatán, México', 492, True, 224511.67794470402, 2314284.761354974)
('53f4b6f3498e823da5166954', 'Unidad básica de rehabilitación Renacimiento', 20.912715, -89.643177, 'México', 383, False, 225065.1977410633, 2314752.3626247244)
('5101870ae4b017638009cad3', 'Healt & Nutrition', 20.925382614135742, -89.68285369873047, 'México', 637, True, 220959.3

In [15]:
print('List of hospitals')
print('---------------------------')
for r in list(hospitals.values())[:10]:
    print(r)
print('...')
print('Total:', len(hospitals))

List of hospitals
---------------------------
('5aa96c8ad7627e7413362a29', 'Villas De Transición Hospitalaria', 20.903318, -89.64109, 'México', 422, True, 225265.2624593384, 2313707.9749038783)
('4dc085c06e8124d25dcf5661', 'Hospital de la Amistad Corea-México', 20.902425049940813, -89.63986104659945, 'Calle 60 (San José Tecoh II), Mérida, Yucatán, México', 655, True, 225391.5431065217, 2313606.967368572)
('52ea95d4498e0eeef0685f29', 'Unidad de Hospitalizacion CIJ', 20.902143, -89.635632, 'México', 232, True, 225831.1848048609, 2313568.4929117775)
('5a136f23a22db7017a9ccf25', 'Consultorio Medico Best', 20.908412, -89.64842, '97297 Mérida, Yucatán, México', 492, True, 224511.67794470402, 2314284.761354974)
('5101870ae4b017638009cad3', 'Healt & Nutrition', 20.925382614135742, -89.68285369873047, 'México', 637, True, 220959.38306904887, 2316224.018416057)
('4fbe4bb6e4b0fb79ac722020', 'San Antonio Xluch', 20.92252550096717, -89.63871403817852, 'México', 506, True, 225547.5466920277, 2315831

In [16]:
#Venue 'Ópticas Devlyn' is incorrectly categorized as "Doctor's Office" since it is an eye care center. 
#We consider that this error in data from source it's not relevant and won't bias our conclusions
version = '20210515'
query = 'Ópticas Devlyn'
test_radius2 = 30000
test_limit = 5
#category = '4bf58dd8d48988d104941735'
test_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&query={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    query,
    #category,
    lat, 
    lon, 
    version, 
    test_radius2, 
    test_limit)
test_url

test_results = requests.get(test_url).json()['response']['groups'][0]['items']
test_results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '50256af321c61aebc69bc8cd',
   'name': 'Ópticas Devlyn',
   'location': {'address': 'Calle 60 # 299 Carretera Merida - Progreso Km 8.5',
    'lat': 21.0395158093409,
    'lng': -89.63091360677139,
    'labeledLatLngs': [{'label': 'display',
      'lat': 21.0395158093409,
      'lng': -89.63091360677139}],
    'distance': 4736,
    'postalCode': '97110',
    'cc': 'MX',
    'city': 'Mérida',
    'state': 'Yucatán',
    'country': 'México',
    'formattedAddress': ['Calle 60 # 299 Carretera Merida - Progreso Km 8.5',
     '97110 Mérida, Yucatán',
     'México']},
   'categories': [{'id': '4bf58dd8d48988d177941735',
     'name': "Doctor's Office",
     'pluralName': "Doctor's Offices",
     'shortName': "Doctor's Office",
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_doctorsoffice_',
      '

In [17]:
print('Medical centers around location')
print('---------------------------')
for i in range(100, 110):
    mc = location_medicals[i][:8]
    names = ', '.join([m[1] for m in mc])
    print('Medical centers around location {}: {}'.format(i+1, names))

Medical centers around location
---------------------------
Medical centers around location 101: 
Medical centers around location 102: 
Medical centers around location 103: Servicio medico industrial
Medical centers around location 104: 
Medical centers around location 105: 
Medical centers around location 106: 
Medical centers around location 107: 
Medical centers around location 108: 
Medical centers around location 109: 
Medical centers around location 110: 


Let's now see all the collected medical centers in our area of interest on a map, and let's also show hospitals in different color.

In [18]:
map_merida = folium.Map(location=merida_center, zoom_start=12)
folium.Marker(merida_center, popup='Merida Centro').add_to(map_merida)
for med in medical_centers.values():
    lat = med[2]; lon = med[3]
    is_hospital = med[6]
    color = 'red' if is_hospital else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_merida)
map_merida

So now we have all the medical centers in area within 12 kilometers from Merida Centro, and we know which ones are hospitals. We also know which medical centers exactly are in vicinity of every neighborhood candidate center.

This concludes the data-gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new hospital!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Merida that have low medical center density, particularly those with low number of hospitals. We will limit our analysis to area ~12km around city center, which covers the entire city and some adjacent inhabited areas.

In first phase we have collected the required **data: location and type (category) of every medical center within 12km from Merida center** (Merida Centro). We have also **identified hospitals** (according to Foursquare categorization and our operational definition).

Second phase in our analysis will be calculation and exploration of '**medical center density**' across different areas of Merida - we will use **heatmaps** to identify a few promising areas around north to center with low number of medical centers in general (*and* no hospitals in vicinity) and focus our attention on those areas.

In third and final phase we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than 4 medical centers in radius of 500 meters**, and we want locations **without hospitals in radius of 500 meters**. We will present a map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of medical centers in every area candidate**:

In [19]:
location_medical_count = [len(med) for med in location_medicals]

df_locations['Medical centers in area'] = location_medical_count

print('Average number of medical centers in every area with radius = 600m:', np.array(location_medical_count).mean())

df_locations.head(10)

Average number of medical centers in every area with radius = 600m: 0.2777777777777778


,Address,Latitude,Longitude,X,Y,Distance from center,Medical centers in area
0,"Diagonal 24, Dzununcán, Mérida, Yucatán, 97315",20.865108,-89.655983,223645.261253,2.309501e+06,11760.689991,0
1,"Mérida, Yucatán",20.865286,-89.644459,224845.261253,2.309501e+06,11450.494709,0
2,"Privada 56, Xmatkuil, Mérida, Yucatán, 97315",20.865464,-89.632935,226045.261253,2.309501e+06,11260.276598,0
3,"Mérida, Yucatán",20.865641,-89.621411,227245.261253,2.309501e+06,11196.152423,0
4,"Mérida, Yucatán",20.865817,-89.609887,228445.261253,2.309501e+06,11260.276598,0
5,"Mérida, Yucatán",20.865993,-89.598363,229645.261253,2.309501e+06,11450.494709,0
6,"Mérida, Yucatán",20.866167,-89.586839,230845.261253,2.309501e+06,11760.689991,0
7,"Mérida, Yucatán",20.874218,-89.673434,221845.261253,2.310541e+06,11503.176225,0
8,"Mérida, Yucatán",20.874398,-89.661910,223045.261253,2.310541e+06,10991.044685,0
9,"Mérida, Yucatán",20.874577,-89.650385,224245.261253,2.310541e+06,10590.706457,0


OK, now let's calculate the **distance to nearest hospital from every area candidate center** (not only those within 600m - we want distance to closest one, regardless of how distant it is).

In [20]:
distances_to_hospital = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for hosp in hospitals.values():
        hosp_x = hosp[7]
        hosp_y = hosp[8]
        d = calc_xy_distance(area_x, area_y, hosp_x, hosp_y)
        if d<min_distance:
            min_distance = d
    distances_to_hospital.append(min_distance)

df_locations['Distance to hospital'] = distances_to_hospital

In [21]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Medical centers in area,Distance to hospital
0,"Diagonal 24, Dzununcán, Mérida, Yucatán, 97315",20.865108,-89.655983,223645.261253,2.309501e+06,11760.689991,0,4461.567451
1,"Mérida, Yucatán",20.865286,-89.644459,224845.261253,2.309501e+06,11450.494709,0,4141.800051
2,"Privada 56, Xmatkuil, Mérida, Yucatán, 97315",20.865464,-89.632935,226045.261253,2.309501e+06,11260.276598,0,4072.771797
3,"Mérida, Yucatán",20.865641,-89.621411,227245.261253,2.309501e+06,11196.152423,0,4305.955595
4,"Mérida, Yucatán",20.865817,-89.609887,228445.261253,2.309501e+06,11260.276598,0,4834.773734
5,"Mérida, Yucatán",20.865993,-89.598363,229645.261253,2.309501e+06,11450.494709,0,5575.734977
6,"Mérida, Yucatán",20.866167,-89.586839,230845.261253,2.309501e+06,11760.689991,0,6176.400829
7,"Mérida, Yucatán",20.874218,-89.673434,221845.261253,2.310541e+06,11503.176225,0,4596.592117
8,"Mérida, Yucatán",20.874398,-89.661910,223045.261253,2.310541e+06,10991.044685,0,3861.056780
9,"Mérida, Yucatán",20.874577,-89.650385,224245.261253,2.310541e+06,10590.706457,0,3273.634526


In [22]:
print('Average distance to closest hospital from each area center:', df_locations['Distance to hospital'].mean())

Average distance to closest hospital from each area center: 1528.8526993296232


OK, so **on average a hospital can be found within ~1500m** from every area center candidate. That seems to be fairly apart even though the average is pulled up by the distances in areas south and outside the city.

Let's create a map showing **heatmap / density of medical centers** and try to extract some meaningfull info from that. Also, let's show few circles indicating distance of 1.5km, 3.0km and 4.5km from Merida Centro as a visual reference.

In [23]:
medical_latlons = [[med[2], med[3]] for med in medical_centers.values()]

hospital_latlons = [[hosp[2], hosp[3]] for hosp in hospitals.values()]

In [24]:
from folium import plugins
from folium.plugins import HeatMap

map_merida = folium.Map(location=merida_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_merida) #cartodbpositron cartodbdark_matter
HeatMap(medical_latlons).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
folium.Circle(merida_center, radius=1500, fill=False, color='white').add_to(map_merida)
folium.Circle(merida_center, radius=3000, fill=False, color='white').add_to(map_merida)
folium.Circle(merida_center, radius=4500, fill=False, color='white').add_to(map_merida)
map_merida

Looks like an interesting pocket of low medical center density north to city center can be found **north-west from Merida Centro**. 

Let's create another heatmap map showing **heatmap/density of hospitals** only.

In [26]:
map_merida = folium.Map(location=merida_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_merida) #cartodbpositron cartodbdark_matter
HeatMap(hospital_latlons).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
folium.Circle(merida_center, radius=1500, fill=False, color='red').add_to(map_merida)
folium.Circle(merida_center, radius=3000, fill=False, color='red').add_to(map_merida)
folium.Circle(merida_center, radius=4500, fill=False, color='red').add_to(map_merida)
map_merida

This map is not so 'hot' (hospitals represent a subset of ~47% of all medical centers in Merida) but it also indicates higher density of existing hospitals directly north from Merida Centro, with biggest pocket of **low hospital density positioned north-west from city center**. Most south side of city is 'cold' meaning low hospital density but this area is not of interest to stakeholders.

Based on this we will now focus our analysis on areas *north-west from Merida center* - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. This places our location candidates mostly in neighborhoods near to the area known as **'Barrio de Chuburna'** and near to neighborhood **Camara de la Construccion**.

### Barrio de Chuburna and Camara de la Construccion

Analysis of some market intelligence sites show that Barrio de Chuburna can be categorized as a 'type C' which can be roughly equivalent to 'middle class' while Camara de la Construccion can be classified as 'type D+' which make it a bit less desirable than the former. Nonetheless, both 'regions of interest' deserve further analysis.

Let's define a new, narrower region of interest, which will include parts of Barrio de Chuburna and Camara de la Construccion neighborhood with low medical centers count north-west to Merida Centro.

In [27]:
roi_x_min = merida_center_x - 3750
roi_y_max = merida_center_y + 8000
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_merida = folium.Map(location=roi_center, zoom_start=13)
HeatMap(medical_latlons).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
folium.Circle(roi_center, radius=2500, color='rose', fill=True, fill_opacity=0.4).add_to(map_merida)
map_merida

This nicely covers the biggest pocket of low medical center density in Barrio de Chuburna and Camara de la Construccion north-west to Merida center.

Let's also create a new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates **200m apart**).

In [28]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 200
y_step = 200 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

565 candidate neighborhood centers generated.


OK. Now let's calculate the two most important things for each location candidate: **number of medical centers in vicinity** (we'll use radius of **500 meters**) and **distance to closest hospital**.

In [29]:
def count_medicals_nearby(x, y, medical_centers, radius=1000):    
    count = 0
    for med in medical_centers.values():
        med_x = med[7]; med_y = med[8]
        d = calc_xy_distance(x, y, med_x, med_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_medical(x, y, medical_centers):
    d_min = 100000
    for med in medical_centers.values():
        med_x = med[7]; med_y = med[8]
        d = calc_xy_distance(x, y, med_x, med_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_medical_counts = []
roi_hospital_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_medicals_nearby(x, y, medical_centers, radius=1000)
    roi_medical_counts.append(count)
    distance = find_nearest_medical(x, y, hospitals)
    roi_hospital_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [30]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Medical centers nearby':roi_medical_counts,
                                 'Distance to hospital':roi_hospital_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Medical centers nearby,Distance to hospital
0,20.993585,-89.636138,225945.261253,2.323698e+06,30,257.007515
1,20.995022,-89.644335,225095.261253,2.323871e+06,19,129.901041
2,20.995052,-89.642413,225295.261253,2.323871e+06,15,115.980142
3,20.995082,-89.640490,225495.261253,2.323871e+06,24,130.628347
4,20.995111,-89.638568,225695.261253,2.323871e+06,27,39.423670
5,20.995141,-89.636646,225895.261253,2.323871e+06,25,199.793286
6,20.995171,-89.634724,226095.261253,2.323871e+06,21,283.856613
7,20.995201,-89.632801,226295.261253,2.323871e+06,20,164.764716
8,20.995231,-89.630879,226495.261253,2.323871e+06,20,231.776286
9,20.995260,-89.628957,226695.261253,2.323871e+06,15,337.737751


Let us now **filter** those locations: we're interested in only in **locations with no more than 4 medical centers in radius of 500 meters**, and **no hospitals in radius of 500 meters**.

In [31]:
good_med_count = np.array((df_roi_locations['Medical centers nearby']<=4))
print('Locations with no more than four medical centers nearby:', good_med_count.sum())

good_hosp_distance = np.array(df_roi_locations['Distance to hospital']>=500)
print('Locations with no hospitals within 500m:', good_hosp_distance.sum())

good_locations = np.logical_and(good_med_count, good_hosp_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]

Locations with no more than four medical centers nearby: 43
Locations with no hospitals within 500m: 242
Locations with both conditions met: 36


Let's see how this looks on a map.

In [32]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_merida = folium.Map(location=roi_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_merida)
HeatMap(medical_latlons).add_to(map_merida)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida) 
map_merida

We now have a bunch of locations north-west to Merida Centro, mostly in **Barrio de Chuburna (Rinconada de Chuburna, San Pedro Uxmal and San Vicente Chuburna)**, **Camara de la Construccion and some parts south from it**. We also know that each of those locations has no more than four medical centers in radius of 500m, and no hospitals closer than 500m. Any of those locations is a potential candidate for a new hospital, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [33]:
map_merida = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida)
map_merida

What we have now is a clear indication of zones with low number of medical centers in vicinity, and *no* hospitals at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [34]:
from sklearn.cluster import KMeans

number_of_clusters = 5

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_merida = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_merida)
HeatMap(medical_latlons).add_to(map_merida)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_merida)
folium.Marker(merida_center).add_to(map_merida)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=400, color='red', fill=True, fill_opacity=0.25).add_to(map_merida) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida)
map_merida

Our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [35]:
map_merida = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(merida_center).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=400, color='red', fill=False).add_to(map_merida) 
map_merida

Let's zoom in on candidate areas in **Rinconada de Chuburna**:

In [36]:
#import geocoder
g_rinconada = geocoder.arcgis('Rinconada de Chuburna, Merida, YUC')
rinconada_latitude = g_rinconada.latlng[0]
rinconada_longitude = g_rinconada.latlng[1]
rinconada_center = rinconada_latitude, rinconada_longitude
print('Rinconada de Chuburna, Merida, Yucatan coordinates are: {}, {}.'.format(g_rinconada.latlng[0], g_rinconada.latlng[1]))
rinconada_center

Rinconada de Chuburna, Merida, Yucatan coordinates are: 21.022620000000074, -89.63472999999999.


(21.022620000000074, -89.63472999999999)

In [37]:
map_merida = folium.Map(location=[21.022620000000074, -89.63472999999999], zoom_start=16)
folium.Marker(merida_center).add_to(map_merida)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=400, color='red', fill=False).add_to(map_merida) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida)
map_merida

...and candidate areas in **San Pedro Uxmal** and **San Vicente Chuburna**:

In [38]:
#import geocoder
g_sn_pedro = geocoder.arcgis('Calle 19 318, San Pedro Uxmal, Merida, YUC')
sn_pedro_latitude = g_sn_pedro.latlng[0]
sn_pedro_longitude = g_sn_pedro.latlng[1]
sn_pedro_center = sn_pedro_latitude, sn_pedro_longitude
print('San Pedro Uxmal, Merida, Yucatan coordinates are: {}, {}.'.format(g_sn_pedro.latlng[0], g_sn_pedro.latlng[1]))
sn_pedro_center

San Pedro Uxmal, Merida, Yucatan coordinates are: 21.01434345357345, -89.64406372175756.


(21.01434345357345, -89.64406372175756)

In [39]:
map_merida = folium.Map(location=[21.01434345357345, -89.64406372175756], zoom_start=16)
folium.Marker(merida_center).add_to(map_merida)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=400, color='red', fill=False).add_to(map_merida) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_merida)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_merida)
map_merida

Finally, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [40]:
#import geocoder
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
#     addr = geocoder.arcgis([lat, lon], method = 'reverse')
    geo_address = geocoder.arcgis([lat, lon], method = 'reverse')
    address = geo_address.address
    candidate_area_addresses.append(address) 
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, merida_center_x, merida_center_y)
    print('{}{} => {:.1f}km from Merida Centro'.format(address, ' '*(80-len(address)), d/1000))

Addresses of centers of areas recommended for further analysis

Calle 60 460, Fracc Camara de la Construcción, Mérida, Yucatán, 97203            => 7.2km from Merida Centro
Calle 17A 101D, Fraccionamiento Chuburna de Hidalgo, Mérida, Yucatán, 97208      => 5.8km from Merida Centro
Calle 21B 308, San Vicente Chuburna, Mérida, Yucatán, 97206                      => 5.4km from Merida Centro
Calle 9 99-99, Fracc Rinconada de Chuburna, Mérida, Yucatán, 97208               => 6.3km from Merida Centro
Calle 29 Diagonal 412, Merida o Elefante Grande, Mérida, Yucatán, 97203          => 6.6km from Merida Centro


This concludes our analysis. We have created **5 addresses** representing centers of zones containing locations with low number of medical centers and no hospitals nearby, all zones being north to city center (all between 5.0km and 7.5km from Merida Centro). Although zones are shown on a map with a radius of ~400 meters (red circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential hospital locations. Most of the zones are located in Barrio de Chuburna and Camara de la Construccion neighborhood, which we have identified as interesting due to being 'middle class' and north to city center.

**We could prioritize our search beginning with the three addresses located in Barrio de Chuburna since it is slightly more interesting than the other ones due to its sociodemographic profile (type 'C').**

In [41]:
map_merida = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(merida_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_merida)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_merida) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_merida)
map_merida

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is an important number of medical centers in Merida, Yucatan (~550 in our initial area of interest, which was 24x24km around Merida Centro), there are pockets of low medical center density north-west and south to city center. Highest concentration of medical was detected exactly north from Merida Centro, so we focused our attention to areas north-west excluding south, corresponding to Barrio de Chuburna (Rinconada de Chuburna, San Pedro Uxmal and San Vincente Chuburna), Camara de la Construccion neighborhood and a couple of areas south of it. 

Something notable was the fact that south of the city and some areas near but outside the city have a really low offer of medical services in general and hospital services in particular. This might be an interesting opportunity for another study and a business opportunity in itself.

After directing our attention to this more narrow area of interest (covering approx. 5x5km north-west from Merida Centro) we first created a dense grid of location candidates (spaced 200m apart); those locations were then filtered so that those with more than 4 medical centers in radius of 500m and those with a hospital closer than 500m were removed.

Those location candidates were then clustered to create zones of interest, which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is five zones containing largest number of potential new medical center locations based on number of and distance to existing venues - both medical centers in general and hospitals particularly. This, of course, does not imply that those zones are actually optimal locations for a new hospital! Purpose of this analysis was to only provide info on areas north to Merida center but not crowded with existing medical centers (particularly hospitals). It is entirely possible that there is a very good reason for small number of medical centers in any of those areas, reasons that would make them unsuitable for a new medical center regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis, which could eventually result in location, which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Merida areas north to center with low number of medical centers (particularly hospitals) in order to aid stakeholders in narrowing down the search for optimal location for a new hospital. By calculating medical center density distribution from Foursquare data we have first identified general areas that justify further analysis (Barrio de Chuburna and Camara de la Construccion), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby medical centers. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal medical center location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to basic services like water, electricity, garbage collection, etc.), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood, etc.